In [2]:
from sklearn.datasets import fetch_20newsgroups
#Choosing categories we require 
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med']
#Will take data from the folder train/test
news_train = fetch_20newsgroups(subset='train',categories=categories,shuffle=True)
news_test = fetch_20newsgroups(subset='test',categories=categories,shuffle=True)

In [3]:
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import BaggingClassifier
import numpy as np

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [4]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', BaggingClassifier(random_state=10)),])

text_mnb_stemmed = text_mnb_stemmed.fit(news_train.data, news_train.target)

predicted_mnb_stemmed = text_mnb_stemmed.predict(news_test.data)

np.mean(predicted_mnb_stemmed == news_test.target)

0.8209054593874834

In [5]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(news_test.target,predicted_mnb_stemmed))
print(metrics.classification_report(news_test.target,predicted_mnb_stemmed,target_names=news_test.target_names)),metrics.confusion_matrix(news_test.target,predicted_mnb_stemmed)

Accuracy: 0.8209054593874834
                        precision    recall  f1-score   support

           alt.atheism       0.83      0.72      0.77       319
         comp.graphics       0.77      0.93      0.84       389
               sci.med       0.84      0.75      0.79       396
soc.religion.christian       0.86      0.86      0.86       398

              accuracy                           0.82      1502
             macro avg       0.82      0.82      0.82      1502
          weighted avg       0.82      0.82      0.82      1502



(None,
 array([[229,  20,  29,  41],
        [  4, 362,  15,   8],
        [ 22,  67, 298,   9],
        [ 20,  22,  12, 344]], dtype=int64))